In [1]:
!pip install -q transformers datasets torch scikit-learn pandas

In [2]:
import pandas as pd
import torch, numpy as np, random, os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [3]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU (slower).")

Using GPU: Tesla T4


In [5]:
from google.colab import files
uploaded = files.upload()

Saving amazon_eco-friendly_products.csv to amazon_eco-friendly_products.csv


In [6]:
df = pd.read_csv("amazon_eco-friendly_products.csv")
print("Rows:", len(df))
df.head()

Rows: 3587


,id,title,name,category,material,brand,price,rating,reviewsCount,description,url,img_url,inStock,inStockText
0,B0CWH366KJ,"Agfabric Natural Jute Erosion Control, 16yard(...",Weed Barrier Fabric,"Patio, Lawn & Garden",NaN,Agfabric,$87.3,NaN,NaN,Protect your yard and garden with our biodegra...,https://www.amazon.com/dp/B0CWH366KJ,https://m.media-amazon.com/images/I/71t3FD5KjH...,True,Only 5 left in stock - order soon.
1,B086L692VC,SAFAVIEH Braided Collection 4' Round Light Blu...,Area Rugs,Home & Kitchen,"50%jute, 25% Wool, 25% Cotton",Safavieh,$40.63,4.2,59.0,Country style is perfect for a casual cottage ...,https://www.amazon.com/dp/B086L692VC,https://m.media-amazon.com/images/I/A1Q73Cheh2...,True,Only 3 left in stock - order soon.
2,B01J6JELTG,Eyeseals 4.0 Sleep Mask – Clear – Moisturizing...,Sleeping Masks,Health & Household,Plastic,EYEECO,$65.95,3.7,1075.0,Locks moisture in: Eyeseals 4.0 eye mask for d...,https://www.amazon.com/dp/B01J6JELTG,https://m.media-amazon.com/images/I/61Uz393xlp...,True,NaN
3,B07HQSKK36,Lucky Monet 25/50/100PCS Burlap Gift Bags Wedd...,Gift Bags,Health & Household,Burlap,Lucky Monet,$29.99,4.6,2492.0,❤ Premium Burlap Material❤ These small burlap ...,https://www.amazon.com/dp/B07HQSKK36,https://m.media-amazon.com/images/I/71DrHIU1aM...,True,In Stock In Stock
4,B0C3Y8WJDR,St. Boniface Bag Company | Burlap Bags - Size:...,Grow Bags,"Patio, Lawn & Garden",5.0 Count,Generic,$29.99,4.4,11.0,100% Burlap > 100% BIODEGRADABLE AND ECO FRIEN...,https://www.amazon.com/dp/B0C3Y8WJDR,https://m.media-amazon.com/images/I/81q3el899U...,True,In Stock


In [7]:
df["text"] = (
    df["title"].astype(str) + " " +
    df["material"].astype(str) + " " +
    df["description"].astype(str)
)

In [8]:
eco_terms = ["eco", "recycl", "sustain", "biodegrad", "organic", "green"]
df["label"] = df["text"].str.contains("|".join(eco_terms), case=False, regex=True).astype(int)

print("Label distribution:")
print(df["label"].value_counts())

Label distribution:
label
1    2841
0     746
Name: count, dtype: int64


In [9]:
train_texts, eval_texts, train_labels, eval_labels = train_test_split(
    df["text"], df["label"], test_size=0.2, stratify=df["label"], random_state=SEED
)
print(f"Train size: {len(train_texts)}, Eval size: {len(eval_texts)}")


Train size: 2869, Eval size: 718


In [13]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from datasets import Dataset

MODEL_NAME = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL_NAME)

def tokenize_function(batch):
    return tokenizer(batch["text"], truncation=True, padding=True, max_length=128)

train_untokenized_ds = Dataset.from_pandas(pd.DataFrame({"text": train_texts, "labels": train_labels}))
eval_untokenized_ds  = Dataset.from_pandas(pd.DataFrame({"text": eval_texts,  "labels": eval_labels}))

tokenized_train = train_untokenized_ds.map(tokenize_function, batched=True)
tokenized_eval  = eval_untokenized_ds.map(tokenize_function,  batched=True)

train_ds = tokenized_train
val_ds = tokenized_eval
num_labels = df["label"].nunique()


Map:   0%|          | 0/2869 [00:00<?, ? examples/s]

Map:   0%|          | 0/718 [00:00<?, ? examples/s]

In [11]:
from transformers import TrainingArguments, Trainer

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    acc = accuracy_score(p.label_ids, preds)
    f1 = f1_score(p.label_ids, preds, average="binary")
    return {"accuracy": acc, "f1": f1}

## **Grid Search Block**

In [19]:
# =====================================================
# 🧩 GRID SEARCH: Exhaustive combination testing
# =====================================================
import os
os.environ["WANDB_DISABLED"] = "true"  # ✅ add this line to skip W&B login
import os, time, itertools, numpy as np, pandas as pd, torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score
from google.colab import files

# ---------- Setup ----------
MODEL_NAME = "distilbert-base-uncased"
device = "cuda" if torch.cuda.is_available() else "cpu"
LOG_PATH = "/content/GridSearch_Experiment_Log.xlsx"

# Dummy check - ensure train_ds, val_ds, num_labels exist before running
assert "train_ds" in globals() and "val_ds" in globals() and "num_labels" in globals(), \
    "Please define train_ds, val_ds, and num_labels first."

# ---------- Parameter Grid ----------
grid_learning_rate = [2e-5, 3e-5, 5e-5]
grid_batch_size    = [8, 16, 32]
grid_epochs        = [2, 3, 4]

# Create Excel header if not exists
if not os.path.exists(LOG_PATH):
    pd.DataFrame(columns=[
        "Run_ID","learning_rate","batch_size","epochs",
        "Accuracy","F1_weighted","Train_Time_s"
    ]).to_excel(LOG_PATH, index=False)

# ---------- Metric Function ----------
def compute_metrics(eval_pred):
    preds = np.argmax(eval_pred.predictions, axis=-1)
    acc = accuracy_score(eval_pred.label_ids, preds)
    f1 = f1_score(eval_pred.label_ids, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}

# ---------- Run All Combinations ----------
combinations = list(itertools.product(grid_learning_rate, grid_batch_size, grid_epochs))
print(f"🔍 Total combinations: {len(combinations)}")

for run_id, (lr, bs, ep) in enumerate(combinations, start=1):
    print(f"\n▶️ Run {run_id}: LR={lr}, BS={bs}, EPOCHS={ep}")
    start = time.time()

    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels).to(device)
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    args = TrainingArguments(
        output_dir=f"/content/run_{run_id}",
        eval_strategy="epoch",
        learning_rate=lr,
        per_device_train_batch_size=bs,
        num_train_epochs=ep,
        weight_decay=0.01,
        logging_dir=f"/content/logs_{run_id}",
        disable_tqdm=True
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer
    )

    trainer.train()
    metrics = trainer.evaluate()
    elapsed = round(time.time() - start, 2)

    row = pd.DataFrame([{
        "Run_ID": run_id,
        "learning_rate": lr,
        "batch_size": bs,
        "epochs": ep,
        "Accuracy": metrics["eval_accuracy"],
        "F1_weighted": metrics["eval_f1"],
        "Train_Time_s": elapsed
    }])
    df = pd.read_excel(LOG_PATH)
    df = pd.concat([df, row], ignore_index=True)
    df.to_excel(LOG_PATH, index=False)

files.download(LOG_PATH)
print("✅ Grid Search completed and logged.")



🔍 Total combinations: 27

▶️ Run 1: LR=2e-05, BS=8, EPOCHS=2


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-3776160938.py:61: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'eval_loss': 0.22539229691028595, 'eval_accuracy': 0.9052924791086351, 'eval_f1': 0.8965555310180436, 'eval_runtime': 2.5714, 'eval_samples_per_second': 279.223, 'eval_steps_per_second': 35.0, 'epoch': 1.0}
{'loss': 0.2564, 'grad_norm': 0.046344418078660965, 'learning_rate': 6.100278551532034e-06, 'epoch': 1.392757660167131}
{'eval_loss': 0.20089547336101532, 'eval_accuracy': 0.9192200557103064, 'eval_f1': 0.9205293483104027, 'eval_runtime': 2.6258, 'eval_samples_per_second': 273.439, 'eval_steps_per_second': 34.275, 'epoch': 2.0}
{'train_runtime': 103.488, 'train_samples_per_second': 55.446, 'train_steps_per_second': 6.938, 'train_loss': 0.23282885219393334, 'epoch': 2.0}
{'eval_loss': 0.20089547336101532, 'eval_accuracy': 0.9192200557103064, 'eval_f1': 0.9205293483104027, 'eval_runtime': 2.5405, 'eval_samples_per_second': 282.621, 'eval_steps_per_second': 35.426, 'epoch': 2.0}

▶️ Run 2: LR=2e-05, BS=8, EPOCHS=3


/tmp/ipython-input-3776160938.py:84: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, row], ignore_index=True)
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-3776160938.py:61: FutureWarning: `tokenizer` is dep

{'eval_loss': 0.28230613470077515, 'eval_accuracy': 0.8788300835654597, 'eval_f1': 0.8609080307753963, 'eval_runtime': 2.5528, 'eval_samples_per_second': 281.261, 'eval_steps_per_second': 35.256, 'epoch': 1.0}
{'loss': 0.2726, 'grad_norm': 0.05804693326354027, 'learning_rate': 1.0733519034354689e-05, 'epoch': 1.392757660167131}
{'eval_loss': 0.19463302195072174, 'eval_accuracy': 0.9192200557103064, 'eval_f1': 0.9205293483104027, 'eval_runtime': 2.5833, 'eval_samples_per_second': 277.943, 'eval_steps_per_second': 34.84, 'epoch': 2.0}
{'loss': 0.1609, 'grad_norm': 5.977817058563232, 'learning_rate': 1.4484679665738164e-06, 'epoch': 2.785515320334262}
{'eval_loss': 0.24896369874477386, 'eval_accuracy': 0.9206128133704735, 'eval_f1': 0.917578038477194, 'eval_runtime': 2.5759, 'eval_samples_per_second': 278.734, 'eval_steps_per_second': 34.939, 'epoch': 3.0}
{'train_runtime': 142.255, 'train_samples_per_second': 60.504, 'train_steps_per_second': 7.571, 'train_loss': 0.2097588169873591, 'epo

...

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-3776160938.py:61: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'eval_loss': 0.1819441020488739, 'eval_accuracy': 0.915041782729805, 'eval_f1': 0.9149360505189911, 'eval_runtime': 2.6284, 'eval_samples_per_second': 273.174, 'eval_steps_per_second': 34.242, 'epoch': 1.0}
{'eval_loss': 0.1566794514656067, 'eval_accuracy': 0.924791086350975, 'eval_f1': 0.9260100829096853, 'eval_runtime': 2.5215, 'eval_samples_per_second': 284.754, 'eval_steps_per_second': 35.693, 'epoch': 2.0}
{'eval_loss': 0.18615204095840454, 'eval_accuracy': 0.9373259052924791, 'eval_f1': 0.9382736970326648, 'eval_runtime': 2.5749, 'eval_samples_per_second': 278.843, 'eval_steps_per_second': 34.952, 'epoch': 3.0}
{'eval_loss': 0.22485998272895813, 'eval_accuracy': 0.9220055710306406, 'eval_f1': 0.920776251724538, 'eval_runtime': 2.5533, 'eval_samples_per_second': 281.202, 'eval_steps_per_second': 35.248, 'epoch': 4.0}
{'train_runtime': 155.2008, 'train_samples_per_second': 73.943, 'train_steps_per_second': 2.32, 'train_loss': 0.14499569998847114, 'epoch': 4.0}
{'eval_loss': 0.2248

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Grid Search completed and logged.


## **Random Search Block**

In [20]:
# =====================================================
# 🎲 RANDOM SEARCH: Random sampling of combinations
# =====================================================
import os, time, random, numpy as np, pandas as pd, torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score
from google.colab import files

# ---------- Setup ----------
MODEL_NAME = "distilbert-base-uncased"
device = "cuda" if torch.cuda.is_available() else "cpu"
LOG_PATH = "/content/RandomSearch_Experiment_Log.xlsx"

# Parameter choices
grid_learning_rate = [2e-5, 3e-5, 5e-5]
grid_batch_size    = [8, 16, 32]
grid_epochs        = [2, 3, 4]

# Create Excel header if not exists
if not os.path.exists(LOG_PATH):
    pd.DataFrame(columns=[
        "Run_ID","learning_rate","batch_size","epochs",
        "Accuracy","F1_weighted","Train_Time_s"
    ]).to_excel(LOG_PATH, index=False)

# ---------- Metric Function ----------
def compute_metrics(eval_pred):
    preds = np.argmax(eval_pred.predictions, axis=-1)
    acc = accuracy_score(eval_pred.label_ids, preds)
    f1 = f1_score(eval_pred.label_ids, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}

# ---------- Random Sampling ----------
num_trials = 5  # <-- Change this to how many random experiments you want
random_combos = [
    (
        random.choice(grid_learning_rate),
        random.choice(grid_batch_size),
        random.choice(grid_epochs)
    )
    for _ in range(num_trials)
]
print(f"🎯 Randomly selected {num_trials} combinations")

for run_id, (lr, bs, ep) in enumerate(random_combos, start=1):
    print(f"\n▶️ Run {run_id}: LR={lr}, BS={bs}, EPOCHS={ep}")
    start = time.time()

    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels).to(device)
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    args = TrainingArguments(
        output_dir=f"/content/random_run_{run_id}",
        eval_strategy="epoch",
        learning_rate=lr,
        per_device_train_batch_size=bs,
        num_train_epochs=ep,
        weight_decay=0.01,
        logging_dir=f"/content/random_logs_{run_id}",
        disable_tqdm=True
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer
    )

    trainer.train()
    metrics = trainer.evaluate()
    elapsed = round(time.time() - start, 2)

    row = pd.DataFrame([{
        "Run_ID": run_id,
        "learning_rate": lr,
        "batch_size": bs,
        "epochs": ep,
        "Accuracy": metrics["eval_accuracy"],
        "F1_weighted": metrics["eval_f1"],
        "Train_Time_s": elapsed
    }])
    df = pd.read_excel(LOG_PATH)
    df = pd.concat([df, row], ignore_index=True)
    df.to_excel(LOG_PATH, index=False)

files.download(LOG_PATH)
print("✅ Random Search completed and logged.")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🎯 Randomly selected 5 combinations

▶️ Run 1: LR=5e-05, BS=8, EPOCHS=2


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-1495017432.py:63: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'eval_loss': 0.18767477571964264, 'eval_accuracy': 0.8913649025069638, 'eval_f1': 0.878368482162278, 'eval_runtime': 3.0642, 'eval_samples_per_second': 234.317, 'eval_steps_per_second': 29.371, 'epoch': 1.0}
{'loss': 0.2461, 'grad_norm': 0.030625449493527412, 'learning_rate': 1.5250696378830085e-05, 'epoch': 1.392757660167131}
{'eval_loss': 0.1765427440404892, 'eval_accuracy': 0.9303621169916435, 'eval_f1': 0.9310259103353753, 'eval_runtime': 2.8056, 'eval_samples_per_second': 255.917, 'eval_steps_per_second': 32.079, 'epoch': 2.0}
{'train_runtime': 146.6898, 'train_samples_per_second': 39.117, 'train_steps_per_second': 4.895, 'train_loss': 0.2239378926481709, 'epoch': 2.0}


/tmp/ipython-input-1495017432.py:86: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, row], ignore_index=True)
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-1495017432.py:63: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'eval_loss': 0.37740209698677063, 'eval_accuracy': 0.8690807799442897, 'eval_f1': 0.8455544386429354, 'eval_runtime': 2.5811, 'eval_samples_per_second': 278.178, 'eval_steps_per_second': 34.869, 'epoch': 1.0}
{'loss': 0.2517, 'grad_norm': 0.021372394636273384, 'learning_rate': 3.2625348189415045e-05, 'epoch': 1.392757660167131}
{'eval_loss': 0.14069446921348572, 'eval_accuracy': 0.9331476323119777, 'eval_f1': 0.9322781697773205, 'eval_runtime': 2.538, 'eval_samples_per_second': 282.904, 'eval_steps_per_second': 35.461, 'epoch': 2.0}
{'loss': 0.1376, 'grad_norm': 0.025354748591780663, 'learning_rate': 1.5215877437325907e-05, 'epoch': 2.785515320334262}
{'eval_loss': 0.2600768506526947, 'eval_accuracy': 0.9331476323119777, 'eval_f1': 0.9328099337535383, 'eval_runtime': 2.5163, 'eval_samples_per_second': 285.341, 'eval_steps_per_second': 35.767, 'epoch': 3.0}
{'eval_loss': 0.3014090657234192, 'eval_accuracy': 0.9345403899721448, 'eval_f1': 0.933953169952717, 'eval_runtime': 2.5553, 'eval

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Random Search completed and logged.
